### 遺產稅試算：

### 114年的新版本

In [1]:
import random
import time

# Selenium 相關模組
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

# Z3 模組
from z3 import Solver, Real, If, And, sat, ToInt

# ---------------------------
# 計算器 function (使用 Z3 求解)
# ---------------------------
def estate_tax_z3_solver(
    # 使用者輸入：死亡日期區間、是否為軍警公教因公死亡
    death_period: int,
    is_military_police: bool,
    
    # 遺產總額相關
    land_value: float,           
    building_value: float,       
    house_value: float,          
    deposit_bonds_value: float,  
    stock_invest_value: float,   
    cash_gold_jewelry_value: float,
    gift_in_2yrs_value: float,   
    
    # 扣除額（人數或金額）
    spouse_count: int,            
    lineal_descendant_count: int,  
    father_mother_count: int,      
    disability_count: int,         
    dependent_count: int,          

    farmland_val: float,           
    inheritance_6to9_val: float,   
    unpaid_tax_fines_val: float,   
    unpaid_debts_val: float,       
    will_management_fee: float,    
    public_facility_retention_val: float,
    spouse_surplus_right_val: float,
    
    # 扣抵稅額
    gift_tax_offset: float,    
    foreign_tax_offset: float  
):
    """
    death_period 對應：
      1 -> 民國103年1月1日 ~ 106年5月11日
      2 -> 民國106年5月12日 ~ 110年12月31日
      3 -> 民國111年1月1日  ~ 112年12月31日
      4 -> 民國113年1月1日(含) ~ 113年12月31日(或113年後之規則)
      5 -> 民國114年1月1日(含)以後

    依據：
    - 免稅額：
      (1)(2) => 12,000,000  
      (3)(4)(5) => 13,330,000  
      若 is_military_police=True => 免稅額加倍 (分別變成 24,000,000 與 26,660,000)。
    
    - 喪葬費：
      (1)(2)(3) => 1,230,000  
      (4)(5)   => 1,380,000

    - 稅率與累進差額：
      (A) death_period = 1：單一稅率 10%，累進差額 0。
      (B) death_period = 2,3,4：
          若課稅遺產淨額 ≤ 50,000,000 元，稅率 10%，累進差額 0；
          若介於 50,000,001 ~ 100,000,000 元，稅率 15%，累進差額 2,500,000；
          若超過 100,000,000 元，稅率 20%，累進差額 7,500,000。
      (C) death_period = 5：
          若課稅遺產淨額 ≤ 56,210,000 元，稅率 10%，累進差額 0；
          若介於 56,210,001 ~ 112,420,000 元，稅率 15%，累進差額 2,810,500；
          若超過 112,420,000 元，稅率 20%，累進差額 8,431,500。
    """

    # (A) 根據 death_period 決定免稅額與喪葬費基準金額
    if death_period in [1, 2]:
        base_exemption_val = 12000000
        funeral_val = 1230000
    else:
        base_exemption_val = 13330000
        funeral_val = 1230000
        if death_period in [4, 5]:
            funeral_val = 1380000

    # 若為軍警公教因公死亡 => 免稅額加倍
    if is_military_police:
        base_exemption_val *= 2

    s = Solver()

    # (B) 建立 Z3 變數
    inheritance_total = Real('inheritance_total')
    base_exemption = Real('base_exemption')
    total_deduction = Real('total_deduction')
    taxable_inheritance = Real('taxable_inheritance')
    estate_tax = Real('estate_tax')
    
    # (C) 遺產總額 = 各項資產之和
    s.add(inheritance_total ==
          land_value + building_value + house_value
          + deposit_bonds_value + stock_invest_value
          + cash_gold_jewelry_value + gift_in_2yrs_value)
    
    # (D) 帶入免稅額與喪葬費
    s.add(base_exemption == base_exemption_val)
    
    # (E) 設定各項扣除額（依據《遺產及贈與稅法第17條》）
    spouse_deduction_val        = Real('spouse_deduction_val')
    lineal_deduction_val        = Real('lineal_deduction_val')
    father_mother_deduction_val = Real('father_mother_deduction_val')
    disability_deduction_val    = Real('disability_deduction_val')
    dependent_deduction_val     = Real('dependent_deduction_val')
    funeral_deduction_val       = Real('funeral_deduction_val')
    
    s.add(
        spouse_deduction_val ==
        spouse_count * If(death_period == 4 or death_period == 5, 5530000, 4930000)
    )
    s.add(
        lineal_deduction_val ==
        lineal_descendant_count * If(death_period == 4 or death_period == 5, 560000, 500000)
    )
    s.add(
        father_mother_deduction_val ==
        father_mother_count * If(death_period == 4 or death_period == 5, 1380000, 1230000)
    )
    s.add(
        disability_deduction_val ==
        disability_count * If(death_period == 4 or death_period == 5, 6930000, 6180000)
    )
    s.add(
        dependent_deduction_val ==
        dependent_count * If(death_period == 4 or death_period == 5, 560000, 500000)
    )
    s.add(funeral_deduction_val == funeral_val)
    
    # (F) 扣除額合計
    s.add(total_deduction ==
          spouse_deduction_val
        + lineal_deduction_val
        + father_mother_deduction_val
        + disability_deduction_val
        + dependent_deduction_val
        + farmland_val
        + inheritance_6to9_val
        + unpaid_tax_fines_val
        + unpaid_debts_val
        + funeral_deduction_val
        + will_management_fee
        + public_facility_retention_val
        + spouse_surplus_right_val
    )

    # (G) 課稅遺產淨額
    s.add(taxable_inheritance == inheritance_total - base_exemption - total_deduction)
    
    # (H) 設定稅率與累進差額
    bracket_rate = Real('bracket_rate')
    bracket_diff = Real('bracket_diff')
    
    s.add(
        If(
            death_period == 1,
            And(bracket_rate == 0.10, bracket_diff == 0),
            If(
                death_period == 5,
                If(
                    taxable_inheritance <= 56210000,
                    And(bracket_rate == 0.10, bracket_diff == 0),
                    If(
                        taxable_inheritance <= 112420000,
                        And(bracket_rate == 0.15, bracket_diff == 2810500),
                        And(bracket_rate == 0.20, bracket_diff == 8431500)
                    )
                ),
                If(
                    taxable_inheritance <= 50000000,
                    And(bracket_rate == 0.10, bracket_diff == 0),
                    If(
                        taxable_inheritance <= 100000000,
                        And(bracket_rate == 0.15, bracket_diff == 2500000),
                        And(bracket_rate == 0.20, bracket_diff == 7500000)
                    )
                )
            )
        )
    )
    
    # (I) 扣抵稅額
    offsets = Real('offsets')
    s.add(offsets == gift_tax_offset + foreign_tax_offset)
    
    # (J) 最終應納遺產稅額：若計算結果為負則取 0
    s.add(
        estate_tax == If(
            taxable_inheritance <= 0,
            0,
            If(taxable_inheritance * bracket_rate - bracket_diff - offsets < 0,
               0,
               taxable_inheritance * bracket_rate - bracket_diff - offsets)
        )
    )
    
    # (K) 求解並轉換為整數
    result = s.check()
    if result == sat:
        model = s.model()
        estate_tax_int = model.evaluate(ToInt(estate_tax))
        return int(estate_tax_int.as_long())
    else:
        return None

### Selenium測試

In [4]:
# ---------------------------
# Selenium 測試程式碼
# ---------------------------
def run_selenium_test():

    # 設定死亡日期下拉選單對應映射 (網頁 option value 與本地 death_period 的對應)
    web_death_mapping = {
        "3": 1,
        "5": 2,
        "6": 3,
        "7": 4,
        "8": 5
    }

    # 測試重複 10 次
    for i in range(10):
        # 隨機產生測試參數
        web_death_value = random.choice(list(web_death_mapping.keys()))
        death_period = web_death_mapping[web_death_value]
        
        # 隨機產生 is_military_police 參數
        is_military_police = random.choice([True, False])
        
        # 隨機產生各項資產
        land_value = random.randint(0, 50000000)
        building_value = random.randint(0, 50000000)
        house_value = random.randint(0, 50000000)
        deposit_bonds_value = random.randint(0, 50000000)
        stock_invest_value = random.randint(0, 50000000)
        cash_gold_jewelry_value = random.randint(0, 50000000)
        gift_in_2yrs_value = random.randint(0, 50000000)
        
        # 隨機產生扣除額參數
        spouse_count = random.randint(0, 1)
        lineal_descendant_count = random.randint(0, 3)
        father_mother_count = random.randint(0, 2)
        disability_count = random.randint(0, 2)
        dependent_count = random.randint(0, 3)
        
        farmland_val = random.randint(0, 10000000)
        inheritance_6to9_val = random.randint(0, 10000000)
        unpaid_tax_fines_val = random.randint(0, 10000000)
        unpaid_debts_val = random.randint(0, 10000000)
        will_management_fee = random.randint(0, 10000000)
        public_facility_retention_val = random.randint(0, 10000000)
        spouse_surplus_right_val = random.randint(0, 10000000)
        
        gift_tax_offset = random.randint(0, 10000000)
        foreign_tax_offset = random.randint(0, 10000000)
        
        # 使用計算器 function 進行本地計算
        local_result = estate_tax_z3_solver(
            death_period, is_military_police,
            land_value, building_value, house_value,
            deposit_bonds_value, stock_invest_value,
            cash_gold_jewelry_value, gift_in_2yrs_value,
            spouse_count, lineal_descendant_count,
            father_mother_count, disability_count,
            dependent_count,
            farmland_val, inheritance_6to9_val,
            unpaid_tax_fines_val, unpaid_debts_val,
            will_management_fee, public_facility_retention_val,
            spouse_surplus_right_val,
            gift_tax_offset, foreign_tax_offset
        )
        
        print(f"\n 第{i+1}次，計算結果: {local_result},death_period: {death_period}, is_military_police: {is_military_police}, land_value: {land_value}, building_value: {building_value}, house_value: {house_value}, deposit_bonds_value: {deposit_bonds_value}, stock_invest_value: {stock_invest_value}, cash_gold_jewelry_value: {cash_gold_jewelry_value}, gift_in_2yrs_value: {gift_in_2yrs_value}, spouse_count: {spouse_count}, lineal_descendant_count: {lineal_descendant_count}, father_mother_count: {father_mother_count}, disability_count: {disability_count}, dependent_count: {dependent_count}, farmland_val: {farmland_val}, inheritance_6to9_val: {inheritance_6to9_val}, unpaid_tax_fines_val: {unpaid_tax_fines_val}, unpaid_debts_val: {unpaid_debts_val}, will_management_fee: {will_management_fee}, public_facility_retention_val: {public_facility_retention_val}, spouse_surplus_right_val: {spouse_surplus_right_val}, gift_tax_offset: {gift_tax_offset}, foreign_tax_offset: {foreign_tax_offset}")

# ---------------------------
# 主程式執行入口
# ---------------------------
if __name__ == "__main__":
    run_selenium_test()



 第1次，計算結果: 13768198,death_period: 5, is_military_police: False, land_value: 18894652, building_value: 46732831, house_value: 47400020, deposit_bonds_value: 35909797, stock_invest_value: 17895018, cash_gold_jewelry_value: 39111677, gift_in_2yrs_value: 42398519, spouse_count: 1, lineal_descendant_count: 1, father_mother_count: 0, disability_count: 2, dependent_count: 1, farmland_val: 3553125, inheritance_6to9_val: 6688609, unpaid_tax_fines_val: 7219114, unpaid_debts_val: 8644870, will_management_fee: 8429435, public_facility_retention_val: 9712373, spouse_surplus_right_val: 8439338, gift_tax_offset: 5102257, foreign_tax_offset: 4785175

 第2次，計算結果: 9315275,death_period: 1, is_military_police: True, land_value: 37080231, building_value: 31407985, house_value: 31380239, deposit_bonds_value: 47871221, stock_invest_value: 18143304, cash_gold_jewelry_value: 20500772, gift_in_2yrs_value: 46493344, spouse_count: 0, lineal_descendant_count: 2, father_mother_count: 2, disability_count: 2, depende